In [1]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import tensorflow as tf

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers.experimental.preprocessing import Rescaling

from tensorflow.keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, BatchNormalization, Activation, Dropout
from tensorflow.keras import Model, Input

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image_dataset_from_directory

In [3]:
DIR = ('/Users/allankapoor/Documents/Springboard/structure_damage/')
train_path = os.path.join(DIR, 'data', 'train_another')
val_path = os.path.join(DIR, 'data', 'validation_another')
test_path = os.path.join(DIR, 'data', 'test_another')

In [5]:
image_size = (128, 128)
batch_size = 100

# specify training image transformations 
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.1,
        zoom_range=0.1,
        horizontal_flip=True)

# specify validation image transformations (rescale only)
val_datagen = ImageDataGenerator(rescale=1./255)

# keras generator for training data
train_generator = train_datagen.flow_from_directory(
        train_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary')

# keras generator for validation data
validation_generator = val_datagen.flow_from_directory(
        val_path,
        target_size=image_size,
        batch_size=batch_size,
        class_mode='binary')

Found 10000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [36]:
# buffered prefetching (?)
train_ds = train_ds.prefetch(buffer_size=32)
val_ds = val_ds.prefetch(buffer_size=32)

In [12]:
def plot_nn_training(history):
    
    '''Given model training history, plots validation accuracy and loss'''
    
    # plot history for accuracy
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()
    
    # plot history for loss
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

In [41]:
def generate_model2(input_shape, num_classes):
    
    '''CNN model architecture with max pooling + dropout layers'''
    
    inputs = Input(shape=input_shape)
    
    # Image augmentation
    #x = data_augmentation(inputs)
    # rescaling
    x = Rescaling(1.0 / 255)(inputs)
    
    # convolution (with max pooling)
    x = Conv2D(32, kernel_size=5, strides=2, padding="same")(x) # put activation in here
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None)(x) 
    x = BatchNormalization()(x) # why?
    x = Activation("relu")(x)

    x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(64, kernel_size=3, strides=2, padding="same")(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None)(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    #flatten
    x = Flatten()(x)
    
    #dense layers (with dropout to avoid overfitting)
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.3)(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.2)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.1)(x)
    
    # output layer
    output = Dense(2, activation='softmax')(x)
    
    mod = Model(inputs=inputs, outputs=output)
    
    return mod

In [42]:
epochs = 50

# patient early stopping
es = EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=5)

mc = ModelCheckpoint('models/Model2/best_model.h5', monitor='val_accuracy', mode='max', verbose=1, save_best_only=True)

callbacks = [tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5")]

# Instantiate model
model2 = generate_model2(input_shape=(128,128,3), num_classes=2)

# Compile model
model2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [ ]:
# train model (not working?)
results2 = model2.fit(
    train_generator,
    steps_per_epoch=100, # num_samples//batch_size
    epochs=50,
    validation_data=validation_generator,
    validation_steps=20) # num_samples//batch_size

In [43]:
# train model
results2 = model2.fit(
    train_ds,
    epochs=50,
    callbacks=callbacks,
    validation_data=val_ds,
)

Epoch 1/50
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
313/313 [==============================] - ETA: 0s - loss: 0.3623 - accuracy: 0.8292WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x14e7fa8b0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSIT

In [8]:
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.applications.resnet50 import ResNet50

In [9]:
def generate_TL_model(input_shape, num_classes):
    
    weights = 'imagenet'
    inputs = Input(input_shape)

    #what does include_top=False remove?
    base_model = ResNet50(include_top=False, weights=weights, input_shape=(128, 128, 3))

    #freezing the resnet layers
    for layer in base_model.layers:
        layer.trainable = False

    #these convolution layers come before the resnet layers
    x = Conv2D(32, (5, 5), strides=2, padding='same', activation='relu', input_shape=(128, 128, 3))(inputs)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None)(x)

    x = Conv2D(64, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None)(x)

    x = Conv2D(64, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid', data_format=None)(x)

    x = Flatten()(x)

    base_resnet = base_model(inputs)
    base_resnet = Flatten()(base_resnet)

    concated_layers = Concatenate()([x, base_resnet])

    # dense layers come after the resnet layers
    # dropout to reduce overfit
    concated_layers = Dense(1024, activation='relu')(concated_layers)
    concated_layers = Dropout(0.3)(concated_layers)
    concated_layers = Dense(512, activation='relu')(concated_layers)
    concated_layers = Dropout(0.2)(concated_layers)
    concated_layers = Dense(256, activation='relu')(concated_layers)
    concated_layers = Dropout(0.1)(concated_layers)
    
    # output layer
    output = Dense(num_classes, activation='softmax')(concated_layers)

    mod = Model(inputs=inputs, outputs=output)
    
    return mod

In [49]:
epochs = 10
callbacks = [tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5")]

# Instantiate model
TL_model = generate_TL_model(input_shape=(128,128,3), num_classes=2)

# Compile model
TL_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [50]:
# train model
TL_model_fit = TL_model.fit(train_ds, epochs=epochs, callbacks=callbacks, validation_data=val_ds)

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
313/313 [==============================] - ETA: 0s - loss: 2.1519 - accuracy: 0.8112WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x17e513160> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSIT

### Now trying with image augmentation:

In [10]:
epochs = 10
callbacks = [tf.keras.callbacks.ModelCheckpoint("save_at_{epoch}.h5")]

# Instantiate model
TL_model_2 = generate_TL_model(input_shape=(128,128,3), num_classes=2)

# Compile model
TL_model_2.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=["accuracy"])

In [11]:
# train model
TL_model_fit_2 = TL_model_2.fit(
    train_generator,
    steps_per_epoch=100, # num_samples//batch_size
    epochs=10,
    validation_data=validation_generator,
    validation_steps=20) # num_samples//batch_size

Epoch 1/10
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


2021-11-27 16:13:54.697454: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-11-27 16:13:54.698013: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


100/100 [==============================] - ETA: 0s - loss: 1.9756 - accuracy: 0.7844WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x3afc90040> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
100/100 [==============================] - 237s 2s/step - loss: 1.9661 - accuracy: 0.7852 - val_loss: 0.3764 - val_accuracy: 0.9300
Epoc